In [4]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
student_scores = pd.read_csv('data/exams.csv')

In [3]:
student_scores.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86


In [25]:
student_scores.isnull().sum()

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

In [27]:
#i want to add an avarage score columns.
student_scores['avarage score']=student_scores[['math score','reading score','writing score']].mean(axis=1)

To perform regression analysis, it is necessary to convert categorical variables into numerical ones. Let's begin by separating the categorical and numerical variables in the dataset.

In [30]:
categorical=student_scores.drop(['math score','reading score', 'writing score','avarage score'], axis=1)
numerical=student_scores[['math score','reading score', 'writing score','avarage score']]

In [32]:
categorical

,gender,race/ethnicity,parental level of education,lunch,test preparation course
0,female,group D,some college,standard,completed
1,male,group D,associate's degree,standard,none
2,female,group D,some college,free/reduced,none
3,male,group B,some college,free/reduced,none
4,female,group D,associate's degree,standard,none
...,...,...,...,...,...
995,male,group C,some college,standard,none
996,male,group C,some college,standard,none
997,female,group A,high school,standard,completed
998,male,group E,high school,standard,none


In [33]:
numerical

,math score,reading score,writing score,avarage score
0,59,70,78,69.000000
1,96,93,87,92.000000
2,57,76,77,70.000000
3,70,70,63,67.666667
4,83,85,86,84.666667
...,...,...,...,...
995,77,77,71,75.000000
996,80,66,66,70.666667
997,67,86,86,79.666667
998,80,72,62,71.333333


In [35]:
df1 = categorical.apply(lambda x: pd.factorize(x)[0])
df1

,gender,race/ethnicity,parental level of education,lunch,test preparation course
0,0,0,0,0,0
1,1,0,1,0,1
2,0,0,0,1,1
3,1,1,0,1,1
4,0,0,1,0,1
...,...,...,...,...,...
995,1,2,0,0,1
996,1,2,0,0,1
997,0,4,5,0,0
998,1,3,5,0,1


In [37]:
data=pd.concat([df1,numerical],axis=1,ignore_index=True)
data

,0,1,2,3,4,5,6,7,8
0,0,0,0,0,0,59,70,78,69.000000
1,1,0,1,0,1,96,93,87,92.000000
2,0,0,0,1,1,57,76,77,70.000000
3,1,1,0,1,1,70,70,63,67.666667
4,0,0,1,0,1,83,85,86,84.666667
...,...,...,...,...,...,...,...,...,...
995,1,2,0,0,1,77,77,71,75.000000
996,1,2,0,0,1,80,66,66,70.666667
997,0,4,5,0,0,67,86,86,79.666667
998,1,3,5,0,1,80,72,62,71.333333


In [38]:
new_columns_name={0:'gender',1:'race',2:'parent education',3:'lunch',4:'preparetion tests',5:'math',6:'reading',7:'writing',8:'avarage'}
data=data.rename(columns=new_columns_name)
data

,gender,race,parent education,lunch,preparetion tests,math,reading,writing,avarage
0,0,0,0,0,0,59,70,78,69.000000
1,1,0,1,0,1,96,93,87,92.000000
2,0,0,0,1,1,57,76,77,70.000000
3,1,1,0,1,1,70,70,63,67.666667
4,0,0,1,0,1,83,85,86,84.666667
...,...,...,...,...,...,...,...,...,...
995,1,2,0,0,1,77,77,71,75.000000
996,1,2,0,0,1,80,66,66,70.666667
997,0,4,5,0,0,67,86,86,79.666667
998,1,3,5,0,1,80,72,62,71.333333


In [39]:
y = data.avarage
X = data.drop(['avarage'], axis=1)

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)

student_scores_model = DecisionTreeRegressor(random_state=1)
student_scores_model.fit(train_X, train_y)

val_predictions = student_scores_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

1.101666666666666


In [51]:
# show the first 5 rows to compare the actual output values with the predicted values
df = pd.DataFrame({'Actual': val_y, 'Predicted': val_predictions})
df1 = df.head(5)
df1

,Actual,Predicted
521,69.333333,70.000000
737,78.000000,77.000000
740,49.666667,46.333333
660,68.666667,67.666667
411,82.666667,83.333333


In [56]:
# get the predicted value for the first row of the validation data
val_X.loc[[740]]

,gender,race,parent education,lunch,preparetion tests,math,reading,writing
740,0,0,5,1,0,39,52,58


In [62]:
# get the actual value for the first row of the validation data
student_scores.loc[[740]]

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,avarage score
740,female,group D,high school,free/reduced,completed,39,52,58,49.666667
